FIBSEM Microscope Testing Notebook

- Testing Basic Functionality
    - Connection to Microscope
    - Image Acquisition
    - Image Data and Metadata 
    

Importing Modules and Utilities built for FIBSEM

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
import fibsem.alignment as fa
from fibsem.structures import BeamType, ImageSettings, GammaSettings, FibsemRectangle, FibsemImage
import matplotlib.pyplot as plt


Setting up connection to Microscope

In [ ]:
# connect to microscope
microscope, microscope_settings = utils.setup_session()

Setting up basic image settings for Ion Beam and Electron Beam
- image settings reduced is for a reduced area with parameters set up with a rectangle

In [ ]:
# set gamma settings
gamma_settings = GammaSettings(
        enabled=True,
        min_gamma=0.5,
        max_gamma=1.8,
        scale_factor=0.01,
        threshold=46,
    )

# set imaging settings
image_settings_eb = ImageSettings(
        resolution="1536x1024",
        dwell_time=1.0e-6,
        hfw=150.0e-6,
        autocontrast=True,
        beam_type=BeamType.ELECTRON,
        gamma=gamma_settings,
        save=False,
        save_path="path",
        label="label",
        reduced_area=None,
    )


image_settings_ib = ImageSettings(
        resolution="1536x1024",
        dwell_time=1.0e-6,
        hfw=150.0e-6,
        autocontrast=True,
        beam_type=BeamType.ION,
        gamma=gamma_settings,
        save=False,
        save_path="path",
        label="label",
        reduced_area=None,
    )
rectangle = FibsemRectangle(0.1, 0.3, 0.4, 0.4)
THERMO_ENABLED = True
image_settings_eb_reduced = ImageSettings(
        resolution="1536x1024",
        dwell_time=1.0e-6,
        hfw=150.0e-6,
        autocontrast=True,
        beam_type=BeamType.ELECTRON,
        gamma=gamma_settings,
        save=False,
        save_path="path",
        label="label",
        reduced_area=rectangle.__to_FEI__() if THERMO_ENABLED else rectangle,
    )



Last image function test

In [ ]:
#testing last image function
img = acquire.last_image(microscope)
plt.imshow(img.data)
plt.show()

Taking set of reference images, now FibsemImages

In [ ]:
#taking set of ref images
hfws =  [400e-6, 150.0e-6]
ref_images = acquire.take_set_of_reference_images(microscope, image_settings_eb,hfws)
fig = plt.figure(figsize=(10, 7))

fig.add_subplot(2, 2, 1)
plt.imshow(ref_images.low_res_eb.data)
fig.add_subplot(2, 2, 2)
plt.imshow(ref_images.high_res_eb.data)
fig.add_subplot(2, 2, 3)
plt.imshow(ref_images.low_res_ib.data)
fig.add_subplot(2, 2, 4)
plt.imshow(ref_images.high_res_ib.data)


assert ref_images.low_res_eb.metadata.image_settings.hfw == hfws[0], f"hfw: {ref_images.low_res_eb.metadata.image_settings.hfw} != {hfws[0]}"
assert ref_images.low_res_ib.metadata.image_settings.hfw == hfws[0], f"hfw: {ref_images.low_res_ib.metadata.image_settings.hfw} != {hfws[0]}"
assert ref_images.high_res_eb.metadata.image_settings.hfw == hfws[1], f"hfw: {ref_images.high_res_eb.metadata.image_settings.hfw} != {hfws[1]}"
assert ref_images.high_res_ib.metadata.image_settings.hfw == hfws[1], f"hfw: {ref_images.high_res_ib.metadata.image_settings.hfw} != {hfws[1]}"


Testing beam alignment using reference image

In [ ]:
new_image = acquire.new_image(microscope, image_settings_eb, None)
fa.align_using_reference_images(microscope, microscope_settings, ref_images.high_res_eb, new_image )
